为了在训练过程中添加进度条，你可以使用 `tqdm` 这个库。它是一个快速、可扩展的Python进度条，可以在 Python 循环或迭代过程中提供美观的进度显示。

### 安装 `tqdm`

首先确保你已经安装了 `tqdm`：

```bash
pip install tqdm
```

### 在训练循环中引入 `tqdm`

接下来，你需要修改你的训练函数来集成 `tqdm`。以下是改进后的代码示例，展示了如何将 `tqdm` 集成到你的 `fit_gpu` 函数中：

```python
from tqdm import tqdm

def fit_gpu(Train_DL, TST_DL, Model_m, epoch_, optim, loss_fn):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    Model_m.to(device)

    correct = 0
    total = 0
    running_loss = 0

    Model_m.train()
    for x, y in tqdm(Train_DL, desc=f'Epoch {epoch_ + 1}/{epochs} Training', unit='batch'):
        x = x.to(device)
        y = y.to(device)

        y_pred = Model_m(x)
        loss = loss_fn(y_pred, y)

        optim.zero_grad()
        loss.backward()
        optim.step()

        with torch.no_grad():
            predicted = torch.argmax(y_pred, dim=1)
            correct += (predicted == y).sum().item()
            total += y.size(0)
            running_loss += loss.item()

    epoch_acc = correct / total
    epoch_loss = running_loss / len(Train_DL.dataset)

    # 测试阶段
    Model_m.eval()
    tstcorrect = 0
    tsttotal = 0
    tstrunning_loss = 0

    with torch.no_grad():
        for x, y in tqdm(TST_DL, desc=f'Epoch {epoch_ + 1}/{epochs} Testing', unit='batch'):
            x, y = x.to(device), y.to(device)
            y_pred = Model_m(x)
            loss = loss_fn(y_pred, y)

            predicted = torch.argmax(y_pred, dim=1)
            tstcorrect += (predicted == y).sum().item()
            tsttotal += y.size(0)
            tstrunning_loss += loss.item()

    tstepoch_acc = tstcorrect / tsttotal
    tstepoch_loss = tstrunning_loss / len(TST_DL.dataset)

    print(
        f'epoch:{epoch_} | loss:{epoch_loss:.3f} | acc:{epoch_acc:.3f} | tstloss:{tstepoch_loss:.3f} | tstacc:{tstepoch_acc:.3f}'
    )

    return epoch_loss, epoch_acc, tstepoch_loss, tstepoch_acc
```

### 代码解释

- **导入 `tqdm`**：从 `tqdm` 库中导入 `tqdm`。
- **包裹数据加载器（DataLoader）**：使用 `tqdm()` 包裹 `Train_DL` 和 `TST_DL`，这样每次迭代都会更新进度条。
- **设置描述和单位**：通过 `desc` 参数设置进度条的描述信息，`unit='batch'` 设置每个单元为 batch。

### 输出效果

当你运行训练脚本时，你会看到类似下面的输出：

```
Epoch 1/10 Training: 100%|██████████| 938/938 [00:15<00:00, 62.47batch/s]
Epoch 1/10 Testing:  100%|██████████| 156/156 [00:02<00:00, 77.90batch/s]
epoch:0 | loss:0.002 | acc:0.951 | tstloss:0.007 | tstacc:0.888
...
```

这不仅使你能够直观地了解训练进度，还能提升调试和观察训练过程的效率。

希望这能帮助你在训练过程中更好地监控进度！如果你有其他问题或需要进一步的帮助，请随时告诉我。 😊

`tqdm` 是一个非常灵活且功能强大的进度条库，适用于 Python 的循环和迭代器。它可以通过简单的接口提供美观的进度显示。下面是一些常用的 `tqdm` 参数及其说明：

### 常用参数

1. **iterable (必需)**:
   - 你希望包装以显示进度的可迭代对象（例如列表、生成器等）。
   - 示例：`for i in tqdm(range(100)): pass`

2. **desc**:
   - 进度条左侧显示的描述信息。
   - 示例：`tqdm(iterable, desc="Processing")`

3. **total**:
   - 总的迭代次数，当 iterable 没有长度时需要指定此参数。
   - 示例：`tqdm(iterable, total=100)`

4. **unit**:
   - 迭代单元的名称，默认为 "it"（iteration）。
   - 示例：`tqdm(iterable, unit="batch")`

5. **unit_scale**:
   - 自动缩放单位到合适的大小（K, M, G, etc.），如数据传输速率等场景中很有用。
   - 示例：`tqdm(iterable, unit_scale=True)`

6. **leave**:
   - 是否在完成迭代后保留进度条，默认为 `True`。
   - 示例：`tqdm(iterable, leave=False)` 如果设置为 `False`，则会在迭代完成后清除进度条。

7. **disable**:
   - 禁用进度条输出，通常用于调试或某些特定条件下不显示进度条的情况。
   - 示例：`tqdm(iterable, disable=True)`

8. **position**:
   - 设置进度条的位置，特别是在嵌套循环中使用多个进度条时非常有用。
   - 示例：`tqdm(iterable, position=0)`

9. **dynamic_ncols**:
   - 动态调整进度条宽度以适应终端窗口大小，默认为 `False`。
   - 示例：`tqdm(iterable, dynamic_ncols=True)`

10. **mininterval**:
    - 更新显示的最小时间间隔（秒），默认为 0.1 秒。
    - 示例：`tqdm(iterable, mininterval=0.5)`

11. **maxinterval**:
    - 更新显示的最大时间间隔（秒），如果超过这个时间没有更新，则会强制刷新进度条。
    - 示例：`tqdm(iterable, maxinterval=10)`

12. **ncols**:
    - 设置进度条的固定宽度（字符数），如果不设置则自动根据终端宽度调整。
    - 示例：`tqdm(iterable, ncols=80)`

13. **ascii**:
    - 使用 ASCII 字符而不是 unicode 来绘制进度条，对于某些旧系统或环境可能有用。
    - 示例：`tqdm(iterable, ascii=True)`

14. **postfix**:
    - 在进度条右侧添加额外的信息，可以是一个字典或者字符串。
    - 示例：`tqdm(iterable, postfix={'loss': 0.01})`

### 示例代码

这里有一个更完整的例子，展示了如何在 PyTorch 训练过程中使用 `tqdm`：

```python
from tqdm import tqdm

def fit_gpu(Train_DL, TST_DL, Model_m, epoch_, optim, loss_fn):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    Model_m.to(device)

    correct = 0
    total = 0
    running_loss = 0

    Model_m.train()
    for x, y in tqdm(Train_DL, 
                     desc=f'Epoch {epoch_ + 1}/{epochs} Training', 
                     unit='batch', 
                     leave=True):
        x = x.to(device)
        y = y.to(device)

        y_pred = Model_m(x)
        loss = loss_fn(y_pred, y)

        optim.zero_grad()
        loss.backward()
        optim.step()

        with torch.no_grad():
            predicted = torch.argmax(y_pred, dim=1)
            correct += (predicted == y).sum().item()
            total += y.size(0)
            running_loss += loss.item()

    epoch_acc = correct / total
    epoch_loss = running_loss / len(Train_DL.dataset)

    # 测试阶段
    Model_m.eval()
    tstcorrect = 0
    tsttotal = 0
    tstrunning_loss = 0

    with torch.no_grad():
        for x, y in tqdm(TST_DL, 
                         desc=f'Epoch {epoch_ + 1}/{epochs} Testing', 
                         unit='batch', 
                         leave=True):
            x, y = x.to(device), y.to(device)
            y_pred = Model_m(x)
            loss = loss_fn(y_pred, y)

            predicted = torch.argmax(y_pred, dim=1)
            tstcorrect += (predicted == y).sum().item()
            tsttotal += y.size(0)
            tstrunning_loss += loss.item()

    tstepoch_acc = tstcorrect / tsttotal
    tstepoch_loss = tstrunning_loss / len(TST_DL.dataset)

    print(
        f'epoch:{epoch_} | loss:{epoch_loss:.3f} | acc:{epoch_acc:.3f} | tstloss:{tstepoch_loss:.3f} | tstacc:{tstepoch_acc:.3f}'
    )

    return epoch_loss, epoch_acc, tstepoch_loss, tstepoch_acc
```

在这个例子中，我们通过 `desc` 参数设置了每个进度条的描述信息，并且通过 `unit='batch'` 来指定迭代的基本单位是 batch。同时，我们还设置了 `leave=True` 以确保进度条在迭代结束后仍然保留在屏幕上。这样可以使训练过程更加直观易懂。